In [1]:
pip install Chetoolbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.0 MB/s eta 0:00:00


In [44]:
import chetoolbox
from chetoolbox import separations
import numpy as np
import numpy.typing as npt
from typing import Callable

**Calculate Boiling Point for a Given Pressure**

In [71]:
def antoine_T(coeffs: npt.NDArray, P: npt.NDArray) -> npt.NDArray:
  '''
  Calculates the temperature in K for each set of vapor pressure in bar.
  '''
  coeffs = np.atleast_2d(coeffs); P = np.c_[np.atleast_1d(P)]
  return (-coeffs[:, 1] / (np.log10(P) - coeffs[:, 0])) - coeffs[:, 2]

In [54]:
antoine_T(coeffs=[4.0266, 1171.53, -48.784], P=[1])

array([[339.73169781]])

**Plot y vs x for a single component**

In [132]:
import numpy as np
import plotly.graph_objects as go

def plot_vle_curve(coeffs_chemical_1, coeffs_chemical_2, Ptot):
    """
    Plots the Vapor-Liquid Equilibrium (VLE) curve for two chemicals using the Antoine equation.

    Parameters:
    coeffs_chemical_1 (list): Antoine constants [A, B, C] for the first chemical.
    coeffs_chemical_2 (list): Antoine constants [A, B, C] for the second chemical.
    Ptot (float): Total system pressure in bar.

    Returns:
    None: Displays the VLE curve plot.
    """

    # Extracting Antoine constants for both chemicals
    A1, B1, C1 = coeffs_chemical_1
    A2, B2, C2 = coeffs_chemical_2

    # Function to calculate boiling point at system pressure (1 bar)
    def antoine_T(coeffs, P):
        A, B, C = coeffs
        return B / (A - np.log10(P)) - C

    # Calculate Boiling Points at System Pressure (1 bar)
    T_bp_chemical_1 = antoine_T([A1, B1, C1], Ptot)
    T_bp_chemical_2 = antoine_T([A2, B2, C2], Ptot)

    # Function to calculate vapor pressure using the Antoine equation
    def antoine_P(A, B, C, T):
        return 10 ** (A - B / (C + T))

    # Temperature range for the plot (from the boiling point of the first chemical to the second one)
    Tplot = np.linspace(T_bp_chemical_1, T_bp_chemical_2, 1000)

    # Calculate vapor pressures for both substances
    Pvap1 = antoine_P(A1, B1, C1, Tplot)
    Pvap2 = antoine_P(A2, B2, C2, Tplot)

    # Calculate mole fractions using Raoult's Law
    xplot = (1 - Pvap2 / Ptot) / (Pvap1 / Ptot - Pvap2 / Ptot)
    yplot = Pvap1 / Ptot * xplot

    # Make sure mole fractions are within valid range [0, 1]
    xplot = np.clip(xplot, 0, 1)
    yplot = np.clip(yplot, 0, 1)

    # Create the plot
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=xplot, y=yplot, mode='lines', name='VLE Curve'))

    fig.update_layout(
        title="Vapor-Liquid Equilibrium Curve",
        xaxis_title="Liquid Mole Fraction (x)",
        yaxis_title="Vapor Mole Fraction (y)",
        width=800,
        height=600,
        template='plotly_dark'
    )

    fig.show()

# Example usage
coeffs_hexane = [4.0266, 1171.53, -48.784]
coeffs_heptane = [4.0282, 1268.636, -56.199]
Ptot = 1  # bar

plot_vle_curve(coeffs_hexane, coeffs_heptane, Ptot)


In [180]:
import numpy as np
import plotly.graph_objects as go

# Antoine constants for Hexane (Lower BP)
A_Lbp = 4.0266
B_Lbp = 1171.53
C_Lbp = -48.784

# Antoine constants for Heptane (Higher BP)
A_Hbp = 4.0282
B_Hbp = 1268.636
C_Hbp = -56.199

# Function to calculate boiling point at system pressure in Kelvin
def antoine_T(coeffs, P):
    A, B, C = coeffs
    T_kelvin = B / (A - np.log10(P)) - C  # Temperature in Kelvin
    T_celsius = T_kelvin - 273.15  # Convert to Celsius
    return T_celsius

# Calculate Boiling Points at System Pressure (1 bar) in Celsius
T_bp_hexane = antoine_T([A_Lbp, B_Lbp, C_Lbp], 1)
T_bp_heptane = antoine_T([A_Hbp, B_Hbp, C_Hbp], 1)

# Function to calculate vapor pressure using the Antoine equation
def antoine_P(A, B, C, T):
    return 10 ** (A - B / (C + T))

# Define system pressure (in bar)
Ptot = 1  # System pressure in bar

# Temperature range for the plot (from the boiling point of Hexane to Heptane)
Tplot = np.linspace(T_bp_hexane, T_bp_heptane, 1000)  # Temperature range

# Calculate vapor pressures for both substances
PvapA = antoine_P(A_Lbp, B_Lbp, C_Lbp, Tplot + 273.15)  # Hexane vapor pressure (convert T to Kelvin for this calculation)
PvapB = antoine_P(A_Hbp, B_Hbp, C_Hbp, Tplot + 273.15)  # Heptane vapor pressure (convert T to Kelvin for this calculation)

# Calculate mole fractions using Raoult's Law
xplot = (1 - PvapB / Ptot) / (PvapA / Ptot - PvapB / Ptot)
yplot = (PvapA / Ptot) * xplot

# Make sure mole fractions are within valid range [0, 1]
xplot = np.clip(xplot, 0, 1)
yplot = np.clip(yplot, 0, 1)

# Plot the equilibrium curve (y vs x)
fig = go.Figure()

fig.add_trace(go.Scatter(x=xplot, y=Tplot, mode='lines', name='Liquid Temperature (T-x)', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=yplot, y=Tplot, mode='lines', name='Vapor Temperature (T-y)', line=dict(color='red')))

# Update layout with the Celsius scale on the y-axis
fig.update_layout(
    title="T-x-y Plot for Mixture",
    xaxis_title="Mole Fraction (x or y)",
    yaxis_title="Temperature (°C)",  # In Celsius
    width=800,
    height=600,
    template='plotly_dark'
)

fig.show()


In [174]:
!pip install chetoolbox
import chetoolbox.separations as che
import chetoolbox.common

In [173]:
che.dew_temp_antoine([0.4, 0.6], [[4.0266, 1171.53, -48.784], [4.0282, 1268.636, -56.199]], 1 )-273.15 #Antoine Coefficients have to be based on K scale, P in Bar

array([88.80189069])

In [175]:
che.bubble_temp_antoine([0.4, 0.6], [[4.0266, 1171.53, -48.784], [4.0282, 1268.636, -56.199]], 1 )-273.15 #Antoine Coefficients have to be based on K scale, P in Bar

array([81.89438406])

In [223]:
import numpy as np
import plotly.graph_objects as go

def plot_relative_volatility_and_K(Ptot, A1, B1, C1, A2, B2, C2):
    """
    Function to plot relative volatility (α) and K-values as a function of temperature.

    Parameters:
    Ptot (float): Total system pressure in bar
    A1, B1, C1 (floats): Antoine coefficients for substance 1
    A2, B2, C2 (floats): Antoine coefficients for substance 2
    """

    # Antoine equation to calculate vapor pressure (P in bar, T in Kelvin)
    def antoine_P(A, B, C, T):
        return np.e ** (A - B / (C + T))

    # Define temperature range in Celsius and convert to Kelvin
    T_C = np.linspace(99, 110, 300)  # Temperature in Celsius
    T_K = T_C + 273.15  # Convert to Kelvin

    # Compute vapor pressures
    Pvap1 = antoine_P(A1, B1, C1, T_C)  # Substance 1
    Pvap2 = antoine_P(A2, B2, C2, T_C)  # Substance 2

    # Compute equilibrium ratios (K-values)
    K1 = Pvap1 / Ptot
    K2 = Pvap2 / Ptot

    # Compute relative volatility α (α = K1/K2)
    alpha = K1 / K2

    # Create plots
    fig = go.Figure()

    # Plot K-values
    fig.add_trace(go.Scatter(x=T_C, y=K1, mode='lines', name='K1 (Substance 1)', line=dict(color='purple')))
    fig.add_trace(go.Scatter(x=T_C, y=K2, mode='lines', name='K2 (Substance 2)', line=dict(color='red')))

    # Plot relative volatility
    fig.add_trace(go.Scatter(x=T_C, y=alpha, mode='lines', name='Relative Volatility (α)', line=dict(color='green', dash='dash')))

    # Layout settings
    fig.update_layout(
        title="Relative Volatility & K-values vs Temperature",
        xaxis_title="Temperature (°C)",
        yaxis_title="K-values & α",
        width=800,
        height=600,
        template='plotly_dark'
    )
    fig.show()
    return


# Example usage:
Ptot = 760  # System pressure in torr
A1, B1, C1 = 15.7831, 2855.27, 213.64  # Antoine Coefficients for n-Heptane
A2, B2, C2 = 17.2741, 3896.3, 255.67  # Antoine Coefficients for Toluene

plot_relative_volatility_and_K(Ptot, A1, B1, C1, A2, B2, C2)
